# Import package

In [1]:
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import gumbel_r
from scipy.stats import gumbel_l
import os
import os.path
import math

from Functions_Indicators_precipitation import changes_in_indicators

# Import data

In [6]:
out_path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6'
csv_path = os.path.join(out_path,'csv_file','pr_day_2021-2060','All_projects_moz_precipitation_2021-2060_without_month_year.csv')
precipitation_2021_2060 = pd.read_csv(csv_path,header=[0,1],index_col=[0,1,2,3,4])*86400 # units mm/day :  1 kg/m2/s = 86400 mm/day
# source: https://www.researchgate.net/post/How-do-I-convert-ERA-Interim-precipitation-estimates-from-kg-m2-s-to-mm-day
list_time_2021_2060=pd.date_range('01-01-2021','31-12-2060', freq='D').strftime('%d-%m-%Y').values.tolist()

In [7]:
precipitation_2021_2060_copy = precipitation_2021_2060.copy(deep=True) # make a coåy that can be modified without changin the original

In [8]:
precipitation_2021_2060_copy

Longitude  \
                                                                    33.625   
Name project          Experiment Model      Date       Latitude              
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125    5.057179   
                                            02-01-2021 -16.125   17.624178   
                                            03-01-2021 -16.125    7.632648   
                                            04-01-2021 -16.125    4.736492   
                                            05-01-2021 -16.125    4.981353   
...                                                                    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125         NaN   
                                            28-12-2060 -13.125         NaN   
                                            29-12-2060 -13.125         NaN   
                                            30-12-2060 -13.125         NaN   
                                            31-12-2060 -13.125         NaN   

                                                                               \
                                                                39.875 36.875   
Name project          Experiment Model      Date       Latitude                 
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125     NaN    NaN   
                                            02-01-2021 -16.125     NaN    NaN   
                                            03-01-2021 -16.125     NaN    NaN   
                                            04-01-2021 -16.125     NaN    NaN   
                                            05-01-2021 -16.125     NaN    NaN   
...                                                                ...    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125     NaN    NaN   
                                            28-12-2060 -13.125     NaN    NaN   
                                            29-12-2060 -13.125     NaN    NaN   
                                            30-12-2060 -13.125     NaN    NaN   
                                            31-12-2060 -13.125     NaN    NaN   

                                                                           
                                                                   40.375  
Name project          Experiment Model      Date       Latitude            
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125        NaN  
                                            02-01-2021 -16.125        NaN  
                                            03-01-2021 -16.125        NaN  
                                            04-01-2021 -16.125        NaN  
                                            05-01-2021 -16.125        NaN  
...                                                                   ...  
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125   0.000000  
                                            28-12-2060 -13.125   0.435942  
                                            29-12-2060 -13.125   6.300328  
                                            30-12-2060 -13.125   8.168522  
                                            31-12-2060 -13.125   0.737792  

[6136200 rows x 4 columns]

In [9]:
# delete model 'NESM3' that does not have any value
precipitation_2021_2060_copy = precipitation_2021_2060.copy(deep=True)
precipitation_2021_2060_copy.drop('NESM3', level=2, axis=0, inplace=True)
precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0],precipitation_2021_2060_copy.index.levels[1][0],'NESM3')]

C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_52732\1858925233.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0],precipitation_2021_2060_copy.index.levels[1][0],'NESM3')]


KeyError: ('Chimoio_2ML_Reservoir', 'ssp126', 'NESM3')

In [10]:
# treatment of dataframe to make useable
df_years=precipitation_2021_2060_copy.copy(deep=True)
#df_year = df_year1.loc[('PT_Revubue_2_Rev_2_01')] # select one project
df_years.columns = df_years.columns.droplevel(0) # drop first level of column name
df_years['33.625'] = df_years['33.625'].fillna(0)
df_years['39.875'] = df_years['39.875'].fillna(0)
df_years['36.875'] = df_years['36.875'].fillna(0)
df_years['40.375'] = df_years['40.375'].fillna(0)
df_years['33.625'] = df_years['33.625'] + df_years['39.875'] + df_years['36.875'] + df_years['40.375']
df_years = df_years[['33.625']]
df_years=df_years.rename(columns={df_years.columns[0]:'Precipitation mm/day period 2021-2060'})
df_years=df_years.droplevel(level=4) # drop latitude index
df_years=df_years.reset_index()

Year = df_years[['Date']].values.reshape(len(df_years[['Date']].values),)
for i in np.arange(0,len(df_years[['Date']].values)):
    Year[i]=int(Year[i][6:10])

df_years['Year'] = Year
df_years

,Name project,Experiment,Model,Date,Precipitation mm/day period 2021-2060,Year
0,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,01-01-2021,5.057179,2021
1,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,02-01-2021,17.624178,2021
2,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,03-01-2021,7.632648,2021
3,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,04-01-2021,4.736492,2021
4,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,05-01-2021,4.981353,2021
...,...,...,...,...,...,...
5727115,PT__Dumping_Site,ssp370,TaiESM1,27-12-2060,0.000000,2060
5727116,PT__Dumping_Site,ssp370,TaiESM1,28-12-2060,0.435942,2060
5727117,PT__Dumping_Site,ssp370,TaiESM1,29-12-2060,6.300328,2060
5727118,PT__Dumping_Site,ssp370,TaiESM1,30-12-2060,8.168522,2060


In [11]:
df_2021_2040 = df_years.loc[df_years['Year'].between(2021,2040)]
df_2021_2040

,Name project,Experiment,Model,Date,Precipitation mm/day period 2021-2060,Year
0,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,01-01-2021,5.057179,2021
1,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,02-01-2021,17.624178,2021
2,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,03-01-2021,7.632648,2021
3,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,04-01-2021,4.736492,2021
4,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,05-01-2021,4.981353,2021
...,...,...,...,...,...,...
5719810,PT__Dumping_Site,ssp370,TaiESM1,27-12-2040,0.184525,2040
5719811,PT__Dumping_Site,ssp370,TaiESM1,28-12-2040,5.674063,2040
5719812,PT__Dumping_Site,ssp370,TaiESM1,29-12-2040,0.471744,2040
5719813,PT__Dumping_Site,ssp370,TaiESM1,30-12-2040,5.973260,2040


In [12]:
df_2041_2060 = df_years.loc[df_years['Year'].between(2041,2060)]
df_2041_2060

,Name project,Experiment,Model,Date,Precipitation mm/day period 2021-2060,Year
7305,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,01-01-2041,12.308317,2041
7306,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,02-01-2041,8.492493,2041
7307,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,03-01-2041,10.244099,2041
7308,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,04-01-2041,30.440408,2041
7309,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,05-01-2041,23.266100,2041
...,...,...,...,...,...,...
5727115,PT__Dumping_Site,ssp370,TaiESM1,27-12-2060,0.000000,2060
5727116,PT__Dumping_Site,ssp370,TaiESM1,28-12-2060,0.435942,2060
5727117,PT__Dumping_Site,ssp370,TaiESM1,29-12-2060,6.300328,2060
5727118,PT__Dumping_Site,ssp370,TaiESM1,30-12-2060,8.168522,2060


# Average yearly precipitation

In [13]:
from Functions_Indicators_precipitation import yearly_avg_pr

In [14]:
# For historical period
title_column = 'Yearly average precipitation mm/year period 2021-2040'
df_years_avg_2021_2040_distribution= yearly_avg_pr(df_2021_2040,title_column)

In [15]:
# For future period
title_column = 'Yearly average precipitation mm/year period 2041-2060'
df_years_avg_2041_2060_distribution= yearly_avg_pr(df_2041_2060,title_column)

In [16]:
# Comparaison between past and future period

In [17]:
changes_past_future_indicator = pd.DataFrame()
changes_past_future_indicator1 = changes_in_indicators(df_years_avg_2021_2040_distribution,df_years_avg_2041_2060_distribution,'Average yearly precipitation')
changes_past_future_indicator=pd.concat([changes_past_future_indicator,changes_past_future_indicator1])

In [18]:
changes_past_future_indicator

Average yearly precipitation  \
                              Change in the median in %   
Chimoio_2ML_Reservoir                         -2.202097   
Mugica_dam_road                               -3.949296   
PT1_Pemba__Breidge_Paquite                    -2.045428   
PT_Nicoadala_F5                               -3.107802   
PT_Revubue_2_Rev_2_01                          1.082005   
PT__Dumping_Site                              -2.344221   
WTP_gate_cnr1                                  1.082005   

                                                         \
                           Change in 10-th percentile %   
Chimoio_2ML_Reservoir                         -1.747428   
Mugica_dam_road                               -3.806494   
PT1_Pemba__Breidge_Paquite                    -1.154481   
PT_Nicoadala_F5                               -6.790416   
PT_Revubue_2_Rev_2_01                         -1.444476   
PT__Dumping_Site                              -1.188254   
WTP_gate_cnr1                                 -1.444476   

                                                         
                           Change in 90-th percentile %  
Chimoio_2ML_Reservoir                          0.562997  
Mugica_dam_road                               -2.197210  
PT1_Pemba__Breidge_Paquite                    -2.357272  
PT_Nicoadala_F5                               -1.974410  
PT_Revubue_2_Rev_2_01                          0.881545  
PT__Dumping_Site                              -2.925441  
WTP_gate_cnr1                                  0.881545

### Seasonal average precipitation

In [19]:
from Functions_Indicators_precipitation import avg_dry_season_precipitation

In mozambique, dry season from April to september. Humid season from October to March

In [20]:
# historic

In [21]:
title_column1 = 'Yearly average precipitation mm/season per year for the period 2021-2040'
pr_2021_2040_dry_season_mean_distribution = avg_dry_season_precipitation(df_2021_2040,title_column1)

C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\Functions_Indicators_precipitation.py:354: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pr_dry_season_mean_distribution=pr_dry_season_mean_distribution.reset_index().drop('Season',axis=1).set_index('Name project')


In [22]:
# futur

In [23]:
title_column1 = 'Yearly average precipitation mm/season per year for the period 2041-2060'
pr_2041_2060_dry_season_mean_distribution = avg_dry_season_precipitation(df_2041_2060,title_column1)

C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\Functions_Indicators_precipitation.py:354: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pr_dry_season_mean_distribution=pr_dry_season_mean_distribution.reset_index().drop('Season',axis=1).set_index('Name project')


In [24]:
pr_2041_2060_dry_season_mean_distribution

Yearly average precipitation mm/season per year for the period 2041-2060  \
                                                                                              count   
Name project                                                                                          
Chimoio_2ML_Reservoir                                                    56.0                         
Mugica_dam_road                                                          56.0                         
PT1_Pemba__Breidge_Paquite                                               56.0                         
PT_Nicoadala_F5                                                          56.0                         
PT_Revubue_2_Rev_2_01                                                    56.0                         
PT__Dumping_Site                                                         56.0                         
WTP_gate_cnr1                                                            56.0                         

                                                                           \
                                  mean        std         min         10%   
Name project                                                                
Chimoio_2ML_Reservoir        76.069421  12.852272   40.298488   59.005105   
Mugica_dam_road             148.783259  19.809934  107.704550  120.591997   
PT1_Pemba__Breidge_Paquite  176.540563  24.675154  112.545617  145.813778   
PT_Nicoadala_F5             315.486927  39.395620  253.270941  262.127148   
PT_Revubue_2_Rev_2_01        25.702344   4.949114   13.166689   19.973013   
PT__Dumping_Site            163.924495  23.033458  104.224709  133.433776   
WTP_gate_cnr1                25.702344   4.949114   13.166689   19.973013   

                                                                
                                   50%         90%         max  
Name project                                                    
Chimoio_2ML_Reservoir        77.393657   91.692715  101.685919  
Mugica_dam_road             149.160700  172.969215  187.026991  
PT1_Pemba__Breidge_Paquite  174.456970  215.753152  230.877777  
PT_Nicoadala_F5             313.436397  367.314841  416.791587  
PT_Revubue_2_Rev_2_01        25.917032   31.110132   40.680932  
PT__Dumping_Site            162.571363  199.081215  211.812041  
WTP_gate_cnr1                25.917032   31.110132   40.680932

In [25]:
# Comparaison between past and future period

In [26]:
changes_past_future_indicator1 = changes_in_indicators(pr_2021_2040_dry_season_mean_distribution,pr_2041_2060_dry_season_mean_distribution,'Average dry season precipitation')
#changes_past_future_indicator=pd.concat([changes_past_future_indicator,changes_past_future_indicator1])
pd.merge(changes_past_future_indicator, changes_past_future_indicator1, left_index=True, right_index=True, how='outer')

Average yearly precipitation  \
                              Change in the median in %   
Chimoio_2ML_Reservoir                         -2.202097   
Mugica_dam_road                               -3.949296   
PT1_Pemba__Breidge_Paquite                    -2.045428   
PT_Nicoadala_F5                               -3.107802   
PT_Revubue_2_Rev_2_01                          1.082005   
PT__Dumping_Site                              -2.344221   
WTP_gate_cnr1                                  1.082005   

                                                         \
                           Change in 10-th percentile %   
Chimoio_2ML_Reservoir                         -1.747428   
Mugica_dam_road                               -3.806494   
PT1_Pemba__Breidge_Paquite                    -1.154481   
PT_Nicoadala_F5                               -6.790416   
PT_Revubue_2_Rev_2_01                         -1.444476   
PT__Dumping_Site                              -1.188254   
WTP_gate_cnr1                                 -1.444476   

                                                         \
                           Change in 90-th percentile %   
Chimoio_2ML_Reservoir                          0.562997   
Mugica_dam_road                               -2.197210   
PT1_Pemba__Breidge_Paquite                    -2.357272   
PT_Nicoadala_F5                               -1.974410   
PT_Revubue_2_Rev_2_01                          0.881545   
PT__Dumping_Site                              -2.925441   
WTP_gate_cnr1                                  0.881545   

                           Average dry season precipitation  \
                                  Change in the median in %   
Chimoio_2ML_Reservoir                              1.126932   
Mugica_dam_road                                   -5.900586   
PT1_Pemba__Breidge_Paquite                        -3.362560   
PT_Nicoadala_F5                                   -4.477743   
PT_Revubue_2_Rev_2_01                             -1.507191   
PT__Dumping_Site                                  -3.542630   
WTP_gate_cnr1                                     -1.507191   

                                                         \
                           Change in 10-th percentile %   
Chimoio_2ML_Reservoir                        -11.658801   
Mugica_dam_road                              -13.040792   
PT1_Pemba__Breidge_Paquite                    -8.509459   
PT_Nicoadala_F5                               -9.723633   
PT_Revubue_2_Rev_2_01                         -1.211725   
PT__Dumping_Site                              -9.968265   
WTP_gate_cnr1                                 -1.211725   

                                                         
                           Change in 90-th percentile %  
Chimoio_2ML_Reservoir                          4.855074  
Mugica_dam_road                               -4.254136  
PT1_Pemba__Breidge_Paquite                    -2.097405  
PT_Nicoadala_F5                                0.382411  
PT_Revubue_2_Rev_2_01                         -7.601198  
PT__Dumping_Site                              -3.117568  
WTP_gate_cnr1                                 -7.601198

In [27]:
changes_past_future_indicator

Average yearly precipitation  \
                              Change in the median in %   
Chimoio_2ML_Reservoir                         -2.202097   
Mugica_dam_road                               -3.949296   
PT1_Pemba__Breidge_Paquite                    -2.045428   
PT_Nicoadala_F5                               -3.107802   
PT_Revubue_2_Rev_2_01                          1.082005   
PT__Dumping_Site                              -2.344221   
WTP_gate_cnr1                                  1.082005   

                                                         \
                           Change in 10-th percentile %   
Chimoio_2ML_Reservoir                         -1.747428   
Mugica_dam_road                               -3.806494   
PT1_Pemba__Breidge_Paquite                    -1.154481   
PT_Nicoadala_F5                               -6.790416   
PT_Revubue_2_Rev_2_01                         -1.444476   
PT__Dumping_Site                              -1.188254   
WTP_gate_cnr1                                 -1.444476   

                                                         
                           Change in 90-th percentile %  
Chimoio_2ML_Reservoir                          0.562997  
Mugica_dam_road                               -2.197210  
PT1_Pemba__Breidge_Paquite                    -2.357272  
PT_Nicoadala_F5                               -1.974410  
PT_Revubue_2_Rev_2_01                          0.881545  
PT__Dumping_Site                              -2.925441  
WTP_gate_cnr1                                  0.881545

## Return period

In [ ]:
from Functions_Indicators_precipitation import value_return_period
from Functions_Indicators_precipitation import dataframe_return_period

In [ ]:
return_period=dataframe_return_period(df_2021_2040)

In [ ]:
return_period

In [ ]:
# dataframe contains statistics about futur value for return period
return_period_stats=return_period.astype(int).groupby(['Name project']).describe(percentiles=[.1, .5, .9]) # astype(int) necessary to have stats min, max and percentiles

In [ ]:
return_period_stats

## N-day event

### 1-day event

In [ ]:
from Functions_Indicators_precipitation import dataframe_1_day_event

In [ ]:
df_1_day_event = dataframe_1_day_event(df_2021_2040)

In [ ]:
df_1_day_event

### 5-day event

In [ ]:
from Functions_Indicators_precipitation import dataframe_n_day_event

In [ ]:
df_5_day_event=dataframe_n_day_event(precipitation_2021_2060_copy,5) # checker si ca joue. a commencer a 3h20

# Exposure level

In [ ]:
# Are project vulnerable ? Exposure level

In [ ]:
# create empty dataframe
#midx = pd.MultiIndex.from_product([df_years_avg_2041_2060_distribution.index.tolist(),precipitation_2021_2060_copy.index.levels[1].tolist(),models],names=['Name project','Experiment', 'Model'])
cols = pd.MultiIndex.from_product([('Exposure level',),('Incremental air temperature change', 'Extreme temperature increase','Incremental rainfall change','Extreme rainfall change')])
ExposureLevel = pd.DataFrame(data = [],
                            index = precipitation_2021_2060_copy.index.levels[0].tolist(),
                            columns = cols)

In [ ]:
ExposureLevel

In [ ]:
ExposureLevel.index.tolist()

In [ ]:
for name_p in ExposureLevel.index.tolist():
    if changes_past_future_avg_yearly_pr.loc[name_p,'Change in the median in %']>20 or (changes_past_future_avg_yearly_pr.loc[name_p,'Change in 10-th percentile %']>50 or changes_past_future_avg_yearly_pr.loc[name_p,'Change in 90-th percentile %']>50):
        ExposureLevel.loc[name_p,('Exposure level','Incremental rainfall change')] = 'High'
    if (changes_past_future_avg_yearly_pr.loc[name_p,'Change in 10-th percentile %']>20 or changes_past_future_avg_yearly_pr.loc[name_p,'Change in 90-th percentile %']>20):
        ExposureLevel.loc[name_p,('Exposure level','Incremental rainfall change')] = 'Medium'    
    if (changes_past_future_avg_yearly_pr.loc[name_p,'Change in 10-th percentile %']<20 and changes_past_future_avg_yearly_pr.loc[name_p,'Change in 90-th percentile %']<20):
        ExposureLevel.loc[name_p,('Exposure level','Incremental rainfall change')] = 'Low'  

In [ ]:
ExposureLevel